A quest to answer questions about my pizza delivery dataset with sqlite
![Domino's](http://www.pizzadelivery.org/Portals/0/ContentImages/xebra_dominos_store.jpg)

First I had to figure out how to create a database from my csv and that took a few hours to get right but I'm glad I did.

In [4]:
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
from sqlalchemy.orm import sessionmaker

path = r'C:\Users\angelddaz\OneDrive\Documents\data_training\data\RawDelData.csv'
disk_engine = create_engine('sqlite:///data.db') # Initializes database with filename 311_8M.db in current directory
connection = disk_engine.connect()

j = 0
index_start = 1

for df in pd.read_csv(path, iterator=True, encoding='utf-8'):
    df['Timestamp'] = pd.to_datetime(df['Timestamp']) # Convert to datetimes, welp this line didn't work haha #TOFIX
    df.index += index_start
    columns = ['Key', 'Date', 'mmdd', 'DayOfTheWeek', 'Area', 'Distance', 'Timestamp', 'Tip', 'OrderAmount', 'TipPercent', \
               'Housing', 'GenderOfTipper', 'Cash/Credit_Tip', 'Cash/Credit_Order', 'PersonWhoDelivered', 'Area(text)', \
               'Latitude', 'Longitude', 'month']
    j+=1
    df.to_sql('data', disk_engine, if_exists='replace')
    index_start = df.index[-1] + 1

df.ix[df['Key'] >= 1303]
#Just to see if this works
df = pd.read_sql_query('SELECT Tip, Key, Housing FROM data WHERE Housing = "Apartment" ', disk_engine)

print pd.read_sql_query('SELECT Count(*) FROM data', disk_engine)
connection.close()

   Count(*)
0      1301


1301 is the correct number of rows in our dataset so it looks like we're all good to go in this sql quest!